++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Q&A RAG with Amazon DocumentDB, LlamaIndex and  Amazon Bedrock

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

## 1. Install and import Libraries 

In [ ]:
# Install required libraries
#!pip3.8 install llama_index.core
#!pip3.8 install llama-index llama_index.vector_stores.awsdocdb
#!pip3.8 install pymongo 
#!pip3.8 install llama-index-embeddings-bedrock 

In [ ]:
# Load the required libraries
import boto3, pprint
from pymongo import MongoClient 
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.awsdocdb import AWSDocDbVectorStore
from llama_index.core.response.notebook_utils import display_response
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.settings import Settings

## 2. Setup Bedrock runtime, embedding model and LLM  model

In [ ]:
#Creating bedrock runtime
bedrock_runtime = boto3.client(service_name='bedrock-runtime', region_name='<region name>')

In [ ]:
#Defining embedding model - amazon.titan-embed-g1-text-02
embed_model=BedrockEmbedding(model="amazon.titan-embed-g1-text-02", client=bedrock_runtime)

In [ ]:
#Defining LLM model - anthropic.claude-instant-v1
claude = Bedrock(credentials_profile_name="default", model="anthropic.claude-instant-v1",client=bedrock_runtime)

In [ ]:
#Settings is a simple singleton object that lives throughout your application. It maintains global settings 
Settings.llm=claude
Settings.embed_model=embed_model

## 3. Loading the data

In [ ]:
#Loading the sample transcript from directory sample-datasets
documents = SimpleDirectoryReader("./sample-datasets/Q1-2024-result-transcript.pdf").load_data()

## 3. Connection setup to DocumentDB

In [ ]:
# Set up a connection to your Amazon DocumentDB (MongoDB compatibility) cluster and creating the database
docdb_client = MongoClient(
"<Amazon DocumentDB database cluster connection string>",
port=27017,
username="<username>",
password="<password>",
retryWrites=False,
tls='true',
tlsCAFile="/home/ec2-user/SageMaker/global-bundle.pem") #Change the path as per your destination


In [ ]:
# Prepare collection and database to store embeddings 
db = docdb_client.ragdemo
collection = db.rag
collection.drop()

## 4. Create the vector search index on collection 

In [ ]:
# Create a HNSW vector search index. You can also create an ivfflat index.
collection.create_index ([("embedding","vector")], 
    vectorOptions= {
        "type": "hnsw", 
        "similarity": "euclidean",
        "dimensions": 1536,
        "m": 16,
        "efConstruction": 64},
    name="vector_index")

## 5. Generate Embedding 

In [ ]:
# Chunking out the data for embedding and create nodes
sentence_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=10)
Settings.text_splitter = sentence_splitter
nodes= sentence_splitter.get_nodes_from_documents(documents)

In [ ]:
# Generate embedding for nodes
for node in nodes:
    node_embedding = embed_model.get_text_embedding(node.get_content(metadata_mode="all"))
    node.embedding = node_embedding

## 6. Define  the vector store and add the embedding to vector store

Below code initialises a DocumentDB Atlas vector store object via the LlamaIndex constructor AWSDocDbVectorStore. 

In [ ]:
#Adding node information with embedding in Amazon Documentdb
vector_store =AWSDocDbVectorStore(docdb_client, db_name="ragdemo", collection_name="rag", index_name="vector_index")
vector_store.add(nodes)

## 7. Q&A system on transcript  

In [ ]:
#Intializing index from the vector store for query interfaces
index = VectorStoreIndex.from_vector_store(vector_store)

Query engine is a query interface that allows you to ask question over your data.

In [ ]:
#Starting a query engine interface on index
query_engine = index.as_query_engine()

In [ ]:
#Question 1
query = "When these results were announced?"
response = query_engine.query(query)
display_response(response)

In [ ]:
#Question 2
query = "How much was AWS revenue in Q1?"
response = query_engine.query(query)
display_response(response)
pprint.pprint(response.response)

In [ ]:
#Question 3
query = "what is the Amazon Q?"
response = query_engine.query(query)
display_response(response)

In [ ]:
#Question 4
query = "What model Bedrock added recently?"
response = query_engine.query(query)
display_response(response)

In [ ]:
### The End